# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [1]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [8]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.8 MB/s eta 0:00:00


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [5]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="elastic-search", #give it a meaningful name,
    embedding=embeddings,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········
OpenAI API key: ··········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [6]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [9]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    None

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [10]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="elastic-search",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [13]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    # Ensure each document has the required 'name' metadata
    for doc in docs:
        if 'name' not in doc.metadata:
            doc.metadata['name'] = 'Unknown Source'  # Default value if 'name' is missing

    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is responsible for serving customers and achieving business objectives in North and South America. The team is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. Their main responsibilities include identifying and pursuing new business opportunities, nurturing existing client relationships, and ensuring customer satisfaction. They also collaborate closely with other departments, such as marketing, product development, and customer support, to deliver high-quality products and services to clients.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [43]:
from urllib.request import urlopen
import json

# Loading new dataset > Covid 19 data
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json"

response = urlopen(url)
data = json.load(response)

In [44]:
# Transform the data into a list of dictionaries
documents = []
for key, value in data.items():
    value['location'] = key
    documents.append(value)

# Save the transformed data to a JSON file
with open("temp.json", "w") as json_file:
    json.dump(documents, json_file)

# Verify the saved JSON data
with open("temp.json", "r") as json_file:
    loaded_data = json.load(json_file)
    print(type(loaded_data))  # Should print <class 'list'>
    print(loaded_data[:5])    # Print first 5 records for inspection

<class 'list'>
[{'continent': 'Asia', 'location': 'AFG', 'last_updated_date': '2024-06-16', 'total_cases': 235214.0, 'new_cases': 0.0, 'new_cases_smoothed': 0.0, 'total_deaths': 7998.0, 'new_deaths': 0.0, 'new_deaths_smoothed': 0.0, 'total_cases_per_million': 5718.965, 'new_cases_per_million': 0.0, 'new_cases_smoothed_per_million': 0.0, 'total_deaths_per_million': 194.462, 'new_deaths_per_million': 0.0, 'new_deaths_smoothed_per_million': 0.0, 'reproduction_rate': None, 'icu_patients': None, 'icu_patients_per_million': None, 'hosp_patients': None, 'hosp_patients_per_million': None, 'weekly_icu_admissions': None, 'weekly_icu_admissions_per_million': None, 'weekly_hosp_admissions': None, 'weekly_hosp_admissions_per_million': None, 'total_tests': None, 'new_tests': None, 'total_tests_per_thousand': None, 'new_tests_per_thousand': None, 'new_tests_smoothed': None, 'new_tests_smoothed_per_thousand': None, 'positive_rate': None, 'tests_per_case': None, 'tests_units': None, 'total_vaccinations

In [49]:
# Editing Chunk Size and Overlap
def metadata_func(record: dict, metadata: dict) -> dict:
    print("Record Type:", type(record))
    if isinstance(record, dict):
        metadata["location"] = record.get("location", "Unknown Location")
        metadata["last_updated_date"] = record.get("last_updated_date", "Unknown Date")
        metadata["continent"] = record.get("continent", "Unknown Continent")
    else:
        print("Unexpected record type:", type(record))
    return metadata

# Configure JSONLoader with the correct schema
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".",  # Use the entire list
    metadata_func=metadata_func,
    text_content=False  # Ensure text_content is False since we're working with dictionaries
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50  # define chunk size and chunk overlap
)

# Load and split documents
docs = loader.load_and_split(text_splitter=text_splitter)

Record Type: <class 'list'>
Unexpected record type: <class 'list'>
Record Type: <class 'list'>
Unexpected record type: <class 'list'>


In [50]:
# Initiating Model + Temp
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="elastic-search",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0.5, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

In [51]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    # Ensure each document has the required 'name' metadata
    for doc in docs:
        if 'name' not in doc.metadata:
            doc.metadata['name'] = 'Unknown Source'  # Default value if 'name' is missing

    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("Which is the least vaccinated country in the world?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the country with the lowest vaccination rate globally?', "2. Can you provide information on the world's least vaccinated nation?", '3. Which country has the lowest number of vaccinated individuals?']


---- Answer ----

I'm sorry, I do not have enough information to determine the least vaccinated country in the world. The context pieces provided do not specify the country names or their vaccination rates. Can you provide more specific information for me to answer your question accurately?


In [57]:
limited_docs = docs[:5]  # otherwise we always exceed the token limit

# Create a new chain function
def create_new_chain():
    return _context | LLM_CONTEXT_PROMPT | llm


chain = create_new_chain()
ans = chain.invoke("Which country was most affected financially from Covid?")
print("---- Answer ----")
print(ans)


chain = create_new_chain()
ans = chain.invoke("Which country had the least vaccination rate?")
print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What was the country that suffered the greatest financial impact from Covid?', '2. Which nation experienced the most significant financial consequences due to Covid?', '3. In terms of financial impact, which country was hit the hardest by Covid?']


---- Answer ----
I don't know which country was most affected financially from Covid, as the context provided does not contain information about the financial impact of Covid on different countries. The context only provides information about the number of cases, deaths, and other related statistics in different countries. Without more specific information about the financial impact of Covid, it is not possible to determine which country was most affected.


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the country with the lowest vaccination rate?', '2. Which nation has the lowest percentage of vaccinated individuals?', '3. Can you suggest a country with a low rate of vaccination?']


---- Answer ----

I am unable to answer that question as the context provided does not specify the country of the retrieved data. Without that information, I am unable to determine which country had the least vaccination rate. 


In [59]:
chain = create_new_chain()
ans = chain.invoke("Which is the most vaccinated country in the world?")
print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the country with the highest vaccination rate globally?', '2. Which nation has the highest percentage of vaccinated population?', '3. Can you suggest the top country in terms of COVID-19 vaccination coverage?']


---- Answer ----

The most vaccinated country in the world is currently the United States, with a total of 951,112,815 vaccinations as of the last updated date of July 4th, 2024. This includes 338,119,516 people who have received at least one dose and 327,967,397 people who are fully vaccinated. The country also has a total of 282,438,401 booster shots administered. The total number of vaccinations per hundred people in the US is 211.29, with 75.11 people vaccinated per hundred and 72.86 people fully vaccinated per hundred. However, it's worth noting that the numbers for other countries may not be accurate as some may not report their data or may not have access to as many vaccines.


In [60]:
chain = create_new_chain()
ans = chain.invoke("country with the most cases?")
print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the country with the highest number of reported cases?', '2. Which nation has the highest count of confirmed cases?', '3. Can you tell me the country with the greatest number of COVID-19 cases?']


---- Answer ----

The country with the most cases is currently China, with a total of 99,361,338 cases as of June 16, 2024. However, it is important to note that this data is from an unknown source and may not be accurate. Other countries with high numbers of cases include Chile in South America with 5,400,569 cases and Kuwait in Asia with 667,290 cases. It is also worth mentioning that New Zealand, a country in Oceania, has a significantly lower number of cases at 2,614,723 as of June 30, 2024. 
